# Trabalho prático - Predição de Spam


## 1) Bibliotecas necessárias 

In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

## 2) Carregar dados do DB